In [ ]:
import requests
import json
import csv

# URL base de la API de Mercadona
url_base = "https://tienda.mercadona.es/api/products/"

# Lista de identificadores comprobados --> identificadores = [63145, 4240, 63147]

# Abrir un archivo CSV para escribir los datos
with open('datos_mercadona.csv', 'w', newline='', encoding='ISO-8859-1') as archivo_csv:
    # Crear un objeto escritor CSV
    escritor_csv = csv.writer(
        archivo_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    # Escribir los encabezados de las columnas en el archivo CSV
    escritor_csv.writerow(
        ['ID', 'Categoría', 'Marca', 'Origen', 'Proveedor', 'Precio', 
         'Descripción', 'Imagen Regular'])
    
    # Iterar sobre los identificadores --> for identificador in identificadores:
    for identificador in range(1000):
        # Construir la URL para el identificador actual
        url = url_base + str(identificador) + "/?lang=es&wh=bcn1"
        try:
            # Hacer una petición HTTP GET a la URL
            response = requests.get(url)
            
            if response.ok:
              # Convertir los datos JSON de la respuesta en un diccionario de Python
              data = json.loads(response.content)

              # Verificar si hay errores en los datos obtenidos
              if 'errors' in data:
                  print("Error: El producto no fue encontrado")
              else:
                  identificador = data['id']
                  categoria_name = data['categories'][0]['name']
                  marca = data['brand']
                  origen = data['origin']
                  proveedor = data['details']['suppliers'][0]['name']
                  precio = data['price_instructions']['unit_price']
                  descripcion = data['details']['description']
                  imagen_ = data['photos'][0]['regular']
                  imagen = imagen_.split('?')[0]
                  
                  # Escribir los datos en el archivo CSV
                  escritor_csv.writerow(
                      [identificador, categoria_name, marca, origen,
                      proveedor, precio, descripcion, imagen])
                  print("Identificador {} extraído y almacenado en el archivo csv.".format(identificador))
            else:
              print("Identificador {} no encontrado".format(identificador))

        except requests.exceptions.RequestException as e:
            # Manejar excepción si la petición falla
            print("Error: No se pudo realizar la petición a la API")

In [5]:
import requests
import json
import csv
import time

url_api = "https://tienda.mercadona.es/api/"
sleep_time = 5  # inicializar la variable sleep_time

def productDataExtractor(delay, product_id):
    url = url_api + "products/" + str(product_id) + "/?lang=es&wh=bcn1"

    try:
        retornData = {}
        response = requests.get(url)

        if response.status_code == 200:
            # Convertir los datos JSON de la respuesta en un dicc de Python
            data = json.loads(response.content)
            retornData['id'] = data['id']     
            retornData['marca'] = data['brand']
            retornData['pvp'] = data['price_instructions']['bulk_price']
            retornData['cantidad'] = data['price_instructions']['unit_size']
            retornData['medida'] = data['price_instructions']['size_format']
            retornData['precio'] = data['price_instructions']['unit_price']
            if data['origin'] is not None:
                retornData['origen'] = data['origin'].replace(' ;', ',')
            else:
                retornData['origen'] = None
            retornData['descripcion'] = data['details']['description']
            retornData['imagen'] = data['photos'][0]['regular'].split('?')[0]
            print(retornData)
        elif response.status_code == 401:
            print("Error 401: Token de autorización no válido")
        elif response.status_code == 404:
            print("Error 404: Producto no encontrado")
        elif response.status_code == 429:
            delay += 1
            print("Error 429: Reintentadolo con delay {}".format(delay))
            time.sleep(delay)
            productDataExtractor(delay, product_id)
        else:
            print("Error desconocido: {}".format(response.status_code))

    except requests.exceptions.RequestException as e:
        # Manejar excepción si la petición falla
        print("Error: No se pudo realizar la petición a la API")
    
    return retornData
    
with open('datos_mercadona.csv', 'w', newline='', encoding='ISO-8859-1') as archivo_csv:
    # Crear un objeto escritor CSV
    escritor_csv = csv.writer(
        archivo_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    # Escribir los encabezados de las columnas en el archivo CSV
    escritor_csv.writerow(['group_id', 'group_name', 'category_id', 'category', 
                           'product_id', 'product_name', 'product_brand', 
                           'product_origin', 'product_pvp', 'product_amount',
                           'product_unit', 'product_price', 'product_description',
                           'product_picture'])

    for menu in range(1, 250):
        # Construir la URL para el identificador actual
        url = url_api + "categories/" + str(menu) + "/?lang=es&wh=bcn1"

        try:
          # Hacer una petición HTTP GET a la URL
          response = requests.get(url)
          if response.ok:
            # Convertir los datos JSON de la respuesta en un diccionario de Python
            data = json.loads(response.content)
            #print(data)
            # Verificar si hay errores en los datos obtenidos
            if 'errors' in data:
                print("Error: El producto no fue encontrado")
            else:
                if data['layout'] == 1:
                    for category in data['categories']:
                        print(category['id'], category['name'])
                        for product in category['products']:
                            #print(product['id'], product['display_name'])
                            productData = productDataExtractor(sleep_time, product['id'])
                            if (len(productData) > 0):
                                escritor_csv.writerow([
                                    data['id'],                 # Group Id
                                    data['name'],               # Group Name
                                    category['id'],             # Category Id
                                    category['name'],           # Category Name
                                    product['id'],              # Product Id
                                    product['display_name'],    # Product Name
                                    productData['marca'],       # Marca
                                    productData['origen'],      # Procedencia
                                    productData['pvp'],         # Precio/ud
                                    productData['cantidad'],    # Cantidad
                                    productData['medida'],      # Unidad medida
                                    productData['precio'],      # Precio
                                    productData['descripcion'], # Descripcion
                                    productData['imagen']       # Imagen
                                ])

        except requests.exceptions.RequestException as e:
            # Manejar excepción si la petición falla
            print("Error: No se pudo realizar la petición a la API")

736 Limpieza cocina
{'id': '43914', 'marca': 'Bosque Verde', 'pvp': '1.93', 'cantidad': 0.75, 'medida': 'l', 'precio': '1.45', 'origen': '', 'descripcion': 'Quitagrasas Bosque Verde perfume Marsella con pistola', 'imagen': 'https://prod-mercadona.imgix.net/images/8bb3bc5e755225dbb4d5691f1e797d2a.jpg'}
{'id': '43906', 'marca': 'Bosque Verde', 'pvp': '2.00', 'cantidad': 0.75, 'medida': 'l', 'precio': '1.50', 'origen': '', 'descripcion': 'Quitagrasas sin espuma Bosque Verde perfume limón con pistola', 'imagen': 'https://prod-mercadona.imgix.net/images/d40e504b952be4019ab430a7ddd3cd85.jpg'}
{'id': '23801', 'marca': 'KH-7', 'pvp': '4.74', 'cantidad': 0.78, 'medida': 'l', 'precio': '3.70', 'origen': None, 'descripcion': 'Quitagrasas KH-7 con pistola', 'imagen': 'https://prod-mercadona.imgix.net/images/dbaefefffea0a3b49d14b913d0506f5b.jpg'}
{'id': '43935', 'marca': 'Bosque Verde', 'pvp': '0.03', 'cantidad': 50.0, 'medida': 'ud', 'precio': '1.30', 'origen': '', 'descripcion': 'Toallitas limpia